In [1]:
from dotenv import load_dotenv
import os 
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import FakeEmbeddings

In [2]:
load_dotenv()
API_KEY = os.getenv('key')
Model = "llama-3.1-70b-versatile"


In [3]:
model = ChatGroq(api_key=API_KEY,model=Model)
model.invoke('what is coding?')

AIMessage(content="Coding, also known as computer programming, is the process of designing, writing, testing, and maintaining the source code of computer programs. This source code is written in one or more programming languages, which are used to communicate with a computer.\n\nThink of coding like writing a recipe for a computer. Just as a recipe provides instructions for a human to follow, a computer program provides instructions for a computer to follow. The computer follows these instructions, step by step, to perform a specific task or set of tasks.\n\nHere's a breakdown of the coding process:\n\n1. **Design**: The programmer defines the problem to be solved and identifies the requirements for the program.\n2. **Writing**: The programmer writes the source code in a programming language, such as Python, Java, or C++.\n3. **Testing**: The programmer tests the program to ensure it works as expected and fixes any errors or bugs.\n4. **Debugging**: The programmer identifies and fixes 

In [4]:
parser = StrOutputParser()
chain = model|parser

In [5]:
chain.invoke('what is coding?')

"**Coding**, also known as computer programming, is the process of designing, writing, testing, and maintaining the source code of computer programs. This source code is written in one or more programming languages, which are used to communicate with a computer and instruct it to perform specific tasks.\n\n**What do coders do?**\n\nCoders, also known as programmers or developers, use programming languages to write code that a computer can understand and execute. Their primary tasks include:\n\n1. **Designing algorithms**: Breaking down complex problems into smaller, manageable parts and creating step-by-step instructions for solving them.\n2. **Writing code**: Using programming languages to write the instructions that a computer can understand.\n3. **Testing and debugging**: Identifying and fixing errors in the code to ensure it works as intended.\n4. **Maintaining and updating**: Modifying existing code to add new features, fix bugs, or improve performance.\n\n**Types of coding**\n\nT

In [6]:
file_loader = PyPDFLoader('AI introduction.pdf')
page = file_loader.load_and_split()
len(page)

9

In [16]:
spliter = RecursiveCharacterTextSplitter(chunk_size = 50,chunk_overlap = 10)
pages = spliter.split_documents(page)


In [9]:
vector_storage = FAISS.from_documents(pages,FakeEmbeddings(size=1352))
retriever = vector_storage.as_retriever()

In [10]:
question_template = """
your a smart bot that answers questions based on the context given to you only.
Return the answer.
context:{context}
question:{question}

"""

In [11]:
prompt = PromptTemplate.from_template(template=question_template)
print(prompt.format(context = ' Here is the context to use',
              question = ' Answer this question based on the context'
              ))


your a smart bot that answers questions based on the context given to you only.
Return the answer and don't make things up.
context: Here is the context to use
question: Answer this question based on the context




In [17]:
result = RunnableParallel(context= retriever,question = RunnablePassthrough())

In [13]:
chain = result |prompt | model | parser

In [14]:
retriever.invoke('Reinforcement learning?')

[Document(metadata={'source': 'AI introduction.pdf', 'page': 0}, page_content='Ryerson University\nWhat is Artificial Intelligence?\nImag e by Gerd Altmann from Pixabay\nGenerally'),
 Document(metadata={'source': 'AI introduction.pdf', 'page': 1}, page_content='our\nhead.\nI\nmean\nthat’s\nwhat\nwe\nhave \nbeen\nwatching\nin\ntelevision\nfor\nyears,\nisn’t\nit?\nAnd\nto\na'),
 Document(metadata={'source': 'AI introduction.pdf', 'page': 7}, page_content='in\nthe\nnear\nfuture\nand \nhow\ndrastically\nit\nis\ngoing\nto\nchange\nand\nupgrade\nthe\nworld\nwe\nlive\nin'),
 Document(metadata={'source': 'AI introduction.pdf', 'page': 5}, page_content='the\ngreatest\nthing\nhumankind\nhas\never\nworked\non.\nAI\nis\nbeing \nused\nin\nimage\nand\nspeech\nrecognition')]

In [19]:
chain.invoke('Reinforcement learning?')

'Reinforcement learning: instance is not labelled and the conclusions are drawn by the machines.'